<a href="https://colab.research.google.com/github/pxm7946/Project4-CancerData/blob/main/Machine_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Start Cleaning Our Data***

In [25]:
# This is the file import and you can upload it when you file run it so you don't have to reload each time.
# from google.colab import files
# uploaded = files.upload()

KeyboardInterrupt: ignored

In [79]:
# Additional Dependeics
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

In [80]:
# Read in the file
df = pd.read_csv('CancerData.txt')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [81]:
# Preview of the DataFrame
df

,Survival months,Age recode with <1 year olds,"Derived AJCC Stage Group, 7th ed (2010-2015)",Sex,CS Schema - AJCC 6th Edition,"Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)",Median household income inflation adj to 2018
0,0082,70-74 years,I,Male,Prostate,Non-Hispanic White,"$75,000+"
1,0061,65-69 years,IIB,Male,Prostate,Non-Hispanic White,"$75,000+"
2,0024,50-54 years,IIA,Female,Breast,Non-Hispanic White,"$75,000+"
3,0063,80-84 years,IA,Female,Breast,Non-Hispanic White,"$75,000+"
4,0045,55-59 years,IA,Female,Breast,Non-Hispanic White,"$75,000+"
...,...,...,...,...,...,...,...
2102525,37,60-64 years,IA,Female,Ovary,Hispanic (All Races),"$40,000 - $44,999"
2102526,32,70-74 years,I,Male,Prostate,Non-Hispanic White,"$40,000 - $44,999"
2102527,27,50-54 years,I,Female,Thyroid,Non-Hispanic White,"$75,000+"
2102528,42,70-74 years,II,Male,Anus,Non-Hispanic White,"$75,000+"


In [82]:
df = df.loc[df['Survival months'] != 'Unknown']

In [83]:
df['Survival months'] = pd.to_numeric(df['Survival months'])
df['Survival months'].dtype

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


dtype('int64')

In [84]:
df.columns

Index(['Survival months', 'Age recode with <1 year olds',
       'Derived AJCC Stage Group, 7th ed (2010-2015)', 'Sex',
       'CS Schema - AJCC 6th Edition',
       'Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)',
       'Median household income inflation adj to 2018'],
      dtype='object')

In [85]:
# Rename the columns in new data frame
df_1 = df.rename(columns={'Survival months':'Survival Months','Age recode with <1 year olds': 'Age Group',
       'Derived AJCC Stage Group, 7th ed (2010-2015)':'Cancer Stage', 'Sex': 'Gender',
       'CS Schema - AJCC 6th Edition': 'Cancer Site',
       'Race and origin recode (NHW, NHB, NHAIAN, NHAPI, Hispanic)':'Race',
       'Median household income inflation adj to 2018':'Median Household Income'})
df_1

,Survival Months,Age Group,Cancer Stage,Gender,Cancer Site,Race,Median Household Income
0,82,70-74 years,I,Male,Prostate,Non-Hispanic White,"$75,000+"
1,61,65-69 years,IIB,Male,Prostate,Non-Hispanic White,"$75,000+"
2,24,50-54 years,IIA,Female,Breast,Non-Hispanic White,"$75,000+"
3,63,80-84 years,IA,Female,Breast,Non-Hispanic White,"$75,000+"
4,45,55-59 years,IA,Female,Breast,Non-Hispanic White,"$75,000+"
...,...,...,...,...,...,...,...
2102525,37,60-64 years,IA,Female,Ovary,Hispanic (All Races),"$40,000 - $44,999"
2102526,32,70-74 years,I,Male,Prostate,Non-Hispanic White,"$40,000 - $44,999"
2102527,27,50-54 years,I,Female,Thyroid,Non-Hispanic White,"$75,000+"
2102528,42,70-74 years,II,Male,Anus,Non-Hispanic White,"$75,000+"


In [86]:
# Drop all rows with missing information
df_clean_data = df_1.dropna(how='any')

In [87]:
#Iterate through rows to identify what columns have null values
for column in df_clean_data.columns:
    print(f"Column {column} has {df_clean_data[column].isnull().sum()} null values")

Column Survival Months has 0 null values
Column Age Group has 0 null values
Column Cancer Stage has 0 null values
Column Gender has 0 null values
Column Cancer Site has 0 null values
Column Race has 0 null values
Column Median Household Income has 0 null values


In [88]:
df_clean_data.columns

Index(['Survival Months', 'Age Group', 'Cancer Stage', 'Gender', 'Cancer Site',
       'Race', 'Median Household Income'],
      dtype='object')

In [89]:
y = df_clean_data.drop(['Age Group','Cancer Stage','Gender','Cancer Site','Race','Median Household Income'],axis=1)
y

,Survival Months
0,82
1,61
2,24
3,63
4,45
...,...
2102525,37
2102526,32
2102527,27
2102528,42


In [90]:
testing_data = df_1.drop('Survival Months', axis=1)
testing_data

,Age Group,Cancer Stage,Gender,Cancer Site,Race,Median Household Income
0,70-74 years,I,Male,Prostate,Non-Hispanic White,"$75,000+"
1,65-69 years,IIB,Male,Prostate,Non-Hispanic White,"$75,000+"
2,50-54 years,IIA,Female,Breast,Non-Hispanic White,"$75,000+"
3,80-84 years,IA,Female,Breast,Non-Hispanic White,"$75,000+"
4,55-59 years,IA,Female,Breast,Non-Hispanic White,"$75,000+"
...,...,...,...,...,...,...
2102525,60-64 years,IA,Female,Ovary,Hispanic (All Races),"$40,000 - $44,999"
2102526,70-74 years,I,Male,Prostate,Non-Hispanic White,"$40,000 - $44,999"
2102527,50-54 years,I,Female,Thyroid,Non-Hispanic White,"$75,000+"
2102528,70-74 years,II,Male,Anus,Non-Hispanic White,"$75,000+"


In [91]:
train_dummies = pd.get_dummies(testing_data)
train_dummies

,Age Group_00 years,Age Group_01-04 years,Age Group_05-09 years,Age Group_10-14 years,Age Group_15-19 years,Age Group_20-24 years,Age Group_25-29 years,Age Group_30-34 years,Age Group_35-39 years,Age Group_40-44 years,Age Group_45-49 years,Age Group_50-54 years,Age Group_55-59 years,Age Group_60-64 years,Age Group_65-69 years,Age Group_70-74 years,Age Group_75-79 years,Age Group_80-84 years,Age Group_85+ years,Cancer Stage_0,Cancer Stage_0a,Cancer Stage_0is,Cancer Stage_I,Cancer Stage_IA,Cancer Stage_IA1,Cancer Stage_IA2,Cancer Stage_IANOS,Cancer Stage_IB,Cancer Stage_IB1,Cancer Stage_IB2,Cancer Stage_IBNOS,Cancer Stage_IC,Cancer Stage_IE,Cancer Stage_IEA,Cancer Stage_IEB,Cancer Stage_II,Cancer Stage_IIA,Cancer Stage_IIA1,Cancer Stage_IIA2,Cancer Stage_IIANOS,...,Cancer Site_ParotidGland,Cancer Site_Penis,Cancer Site_Peritoneum,Cancer Site_Placenta,Cancer Site_Pleura,Cancer Site_Prostate,Cancer Site_Rectum,Cancer Site_RenalPelvis,Cancer Site_Scrotum,Cancer Site_Skin,Cancer Site_SkinEyelid,Cancer Site_SmallIntestine,Cancer Site_SoftPalate,Cancer Site_SoftTissue,Cancer Site_Stomach,Cancer Site_SubLarynx,Cancer Site_SubmandibularGland,Cancer Site_SupraLarynx,Cancer Site_Testis,Cancer Site_Thyroid,Cancer Site_Urethra,Cancer Site_Vagina,Cancer Site_Vulva,Race_Hispanic (All Races),Race_Non-Hispanic American Indian/Alaska Native,Race_Non-Hispanic Asian or Pacific Islander,Race_Non-Hispanic Black,Race_Non-Hispanic Unknown Race,Race_Non-Hispanic White,"Median Household Income_$35,000 - $39,999","Median Household Income_$40,000 - $44,999","Median Household Income_$45,000 - $49,999","Median Household Income_$50,000 - $54,999","Median Household Income_$55,000 - $59,999","Median Household Income_$60,000 - $64,999","Median Household Income_$65,000 - $69,999","Median Household Income_$70,000 - $74,999","Median Household Income_$75,000+","Median Household Income_< $35,000",Median Household Income_Unknown/missing/no match/Not 1990-2017
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2102525,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2102526,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
2102527,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2102528,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0


### ***Start of Machine Learning***

In [ ]:
# Assign the data to X and y
# Note: Sklearn requires a two-dimensional array of values
# so we use reshape() to create this

X = notes[['variance','skewness','curtosis', 'entropy']]
y = notes['class']

print("Shape: ", X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))
train_df.head()